# Redundant Capacity Optimization System
## Professional Resource Management & Assignment Optimization

### Project Overview
This system optimizes the allocation of medical abstractors across different projects by identifying opportunities to redistribute experienced personnel to high-priority assignments while backfilling their previous positions with qualified trainees. The primary goal is to maximize the efficiency of resource utilization while ensuring all projects maintain appropriate skill coverage.

### Business Context
In healthcare data abstraction, experienced abstractors often possess multiple specialized skills, while newer team members (trainees) may have more limited skill sets. This creates an opportunity to optimize team deployment by:
1. Identifying projects where experienced abstractors are working below their full skill capacity
2. Relocating these abstractors to projects that require their advanced skill sets
3. Backfilling their original positions with qualified trainees

### Technical Implementation
The system implements this optimization through several key components:

1. **Resource Analysis**
   - Evaluates current assignments and abstractor skill sets
   - Identifies opportunities where experienced abstractors could be better utilized
   - Calculates available capacity and required hours for each project

2. **Scoring Algorithm**
   - Ranks potential moves based on multiple factors:
     - Full-time vs. part-time status
     - Current workload
     - Existing team assignments
     - Skill match requirements
     - Previous assignment history

3. **Assignment Optimization**
   - Generates optimal reassignment recommendations
   - Ensures all projects maintain required skill coverage
   - Validates that proposed changes meet business rules and constraints

4. **Integration**
   - Connects with Salesforce for real-time data
   - Utilizes Excel for input/output of assignment plans
   - Maintains tracking of historical movements and capacity changes

### Technical Stack
- Python (pandas, numpy)
- Salesforce Integration (simple_salesforce)
- Excel Integration (openpyxl)
- Custom scoring and optimization algorithms

### Output
The system produces detailed reports showing:
- Recommended resource movements
- Backfill recommendations
- Capacity impact analysis
- Scoring breakdowns for all considered options

# Technical Code Summary

## Core Components

### Data Sources
- Salesforce database (via simple_salesforce)
- Excel files containing:
  - Current capacity base
  - Hours already shifted
  - Daily assignment planning

### Key Functions

1. `check_for_existing_assignment()`
   - Validates if an abstractor is already assigned to a team
   - Prevents single-resource teams from being disrupted
   - Returns scoring modifications based on assignment status

2. `score_row()`
   - Implements the scoring algorithm for potential moves
   - Considers factors such as:
     - Full/Part time status (+0/+4 points)
     - Hours match proximity (+2 points if within 0.5 hours)
     - Existing team assignment (+10/-10 points)
     - Previous shifts (-10 points if already being shifted)

### Main Process Flow

1. **Initial Setup**
   - Loads configuration and current assignments
   - Filters for relevant categories and skills
   - Establishes scoring criteria

2. **Capacity Analysis**
   - Identifies abstractors with potential for movement
   - Calculates available hours and needed coverage
   - Filters based on minimum working hours (2+ hours in 4 or 12 week periods)

3. **Optimization Loop**
   - For each target category:
     - Scores potential moves
     - Identifies optimal reassignments
     - Calculates backfill requirements
     - Updates capacity tracking

4. **Output Generation**
   - Creates Excel workbook with multiple sheets:
     - Distribution recommendations
     - Distribution options
     - Backfill options
   - Maintains tracking file of processed movements

### Key Features
- Prevents disruption of critical assignments
- Maintains skill coverage requirements
- Tracks and prevents double-booking of resources
- Provides multiple options for each recommended move
- Generates comprehensive documentation of decisions

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import os
from datetime import datetime, timedelta
from simple_salesforce import Salesforce
# Import the module under a specific name
import importlib
import sf_queries_class
importlib.reload(sf_queries_class)
from sf_queries_class import SfQueries
import my_sf_secrets
import helper_fx
import capacity_portion
importlib.reload(capacity_portion)
import create_capbase
importlib.reload(create_capbase)
my_sf_username, my_sf_password, my_sf_security_token = my_sf_secrets.get_my_sf_secrets()
queries = SfQueries(
    username=my_sf_username,
    password=my_sf_password,
    security_token=my_sf_security_token
)

# get the username from the system
username = os.getlogin()
print(f'{filename}')
from reportforce import Reportforce
rf = Reportforce(session_id=queries.sf.session_id, instance_url=queries.sf.sf_instance)

capbase = pd.read_excel(filename).fillna({'current_skills': ""})
assignment_hours_already_shifted = pd.read_excel('hours_already_shifted.xlsx')
todays_needs = pd.read_excel(f"Daily Tools/Assignment_Planning_{today}.xlsx", sheet_name='Summary')
todays_needs_fil = todays_needs[todays_needs['Suggested Resource'].isnull()].copy()

In [ ]:
cats_with_capacity_to_shift = ('cat1', 'cat2')
pci_sts_acs = capacity_portion.category_capacity(list(cats_with_capacity_to_shift), capbase, capacity_filter=False)
sorted(pci_sts_acs.columns)

In [3]:
def check_for_existing_assignment(CBIZ_Name, request_names, risk_mitigation_resource):
    team_ids = []
    for request_name in request_names:
        team_id = queries.get_sr_from_request_name(request_name)['ID'].values[0]
        team_ids.append(team_id)
    abstractor_id = queries.get_contact_id_from_CBIZ_Name(CBIZ_Name)
    team_members = queries.get_abstractor_assignments(team_id=team_ids[0])
    team_members_abs = team_members[(team_members['Team_Position__c'] == 'Abstractor') &
                                    (team_members['Resource__r.Name'] != risk_mitigation_resource)]
    if (risk_mitigation_resource is not None) and (risk_mitigation_resource is not np.nan):
        if (abstractor_id == queries.get_contact_id_from_CBIZ_Name(risk_mitigation_resource)):
            return -10
    if (len(team_members_abs) <= 1) & (abstractor_id in list(team_members_abs['Id'])):
        return -10
    elif abstractor_id in list(team_members_abs['Id']):
        return 10
    else:
        return 0


In [4]:
def score_row(row, cap_col_name, hours_needed, request_name, risk_mitigation_resource, already_shifting=None):
    score = 0
    
    # Score based on Full Time Status
    if row['Full Time Status'] == 'PT':
        score += 4
    elif row['Full Time Status'] == 'FT':
        score += 0
    
    # Score based on proximity to hours_needed
    if abs(row[cap_col_name] - hours_needed) <= 0.5:
        score += 2
    
    # Add to Score if already on team
    score += check_for_existing_assignment(row['CBIZ_Name'], request_name, risk_mitigation_resource[0])

    # if the abstractor is already being shifted, reduce the score by 1
    if already_shifting is not None:
        if (row['CBIZ_Name'].split(' - ')[0] in already_shifting) or (row['CBIZ_Name'] in already_shifting):
            # print(score)
            score -= 10
            # print(f"abstractor: {row['CBIZ_Name']} is already shifting {score}")
    
    # Score based on row position
    # score += (1 / (index + 1))
    
    return score

In [ ]:
todays_needs_fil['Project Category'].unique()

In [ ]:
# create a function that will check if any of them are already on the project that has a staffing request, 
# and if they are the only one, if not, then give them the highest ranking
assignment_hours_already_shifted = pd.read_excel('hours_already_shifted.xlsx')
cats_with_capacity_to_shift = ('cat1', 'cat2')
pci_sts_acs = capacity_portion.category_capacity(list(cats_with_capacity_to_shift), capbase, capacity_filter=False)


writer = pd.ExcelWriter(f"Daily Tools/Redundant_Capacity_Planning_{today}.xlsx", engine='xlsxwriter')
requests_to_exclude = ['specific requests']
pci_abstractors_to_exclude = ['specific abstractors']

target_categories = todays_needs_fil['Project Category'].unique()
todays_needs_fil = todays_needs_fil[~todays_needs_fil['Request Name'].isin(requests_to_exclude)]
# group by team name and sum the Hours Requested column
# and aggregate 'Request Name' and 'Risk Mitigation Resource: Full Name' into lists
todays_needs_fil_gr = todays_needs_fil.groupby(['Team Name', 'Project Category']) \
    .agg({'Request Name': list, 'Risk Mitigation Resource: Full Name': list, 'Requested Hours': 'sum'}) \
    .reset_index()

# Concatenate the lists of 'Request Name' and 'Risk Mitigation Resource: Full Name' 
# into a single string for each group
todays_needs_fil_gr['Combined Names'] = todays_needs_fil_gr.apply(
    lambda row: ', '.join(f"{req} - {name}" if name else req for req, name in zip(
        row['Request Name'], row['Risk Mitigation Resource: Full Name'])), axis=1
)

for target_category in ['TC1', 'TC2']:
    redundant_capacity_df, final_redundant_capacity_df, all_redundant_options_df, all_backfill_options_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    if target_category == 'Specific category 1':
        target_category_skills = 'SC1'
    elif 'Specific category 2' in target_category:
        target_category_skills = 'Specific category 2'
    else:
        target_category_skills = target_category
    

    pci_sts_acs_sel = pci_sts_acs[(~pci_sts_acs.CBIZ_Name.isin(pci_abstractors_to_exclude)) &
                            (pci_sts_acs['current_skills'].str.contains(target_category_skills))].copy()
    # print(pci_sts_acs_sel)
    if len(pci_sts_acs_sel) > 0: 
        target_cat_abstractor_assignments = queries.get_all_assignments(pci_sts_acs_sel, 'CBIZ_Name')
        target_cat_abstractor_assignments_real = target_cat_abstractor_assignments[
            ((target_cat_abstractor_assignments['Project_Category__c'].str.contains(cats_with_capacity_to_shift[0])) | 
             (target_cat_abstractor_assignments['Project_Category__c'].str.contains(cats_with_capacity_to_shift[1]))) & 
                                    ((target_cat_abstractor_assignments['Wrkd_12__c'] >= 2) |
                                    (target_cat_abstractor_assignments['Wrkd_4__c'] >= 2)) & 
                                    (~target_cat_abstractor_assignments['Team_Position__c'].isin(['QAR', 'Temporary Resource', 'Backup'])) &
                                     (target_cat_abstractor_assignments['Project_Not_Eligible_for_Trainee__c'] == False) &
                                     (~target_cat_abstractor_assignments['Id'].isin(assignment_hours_already_shifted['id'].values))
                                    ].copy()
        target_cat_abstractor_assignments_real.loc[:, 'CBIZ_Name'] = target_cat_abstractor_assignments_real['Resource__r.Name'] + ' - ' + \
            target_cat_abstractor_assignments_real['Resource__r.External_Employee_ID__c'].astype(int).astype(str)

        target_cat_abstractor_assignments_real_merged = target_cat_abstractor_assignments_real.rename(
            columns={'Resource__r.Full_Time_Status__c': 'Full Time Status'}
            ).sort_values(by=['Full Time Status', 'Wrkd_12__c'], ascending=[False, False])

        hours_needed_in_target_category =  sum(todays_needs_fil_gr.loc[
            (todays_needs_fil_gr['Project Category'] == target_category), 'Requested Hours'])
        target_category_requests = todays_needs_fil_gr.loc[
            (todays_needs_fil_gr['Project Category'] == target_category), ['Team Name', 'Requested Hours', 
                                                                             'Request Name', 'Risk Mitigation Resource: Full Name']]
        # print(hours_needed_in_target_category)
        if len(target_cat_abstractor_assignments_real_merged) > 0:
            iteration = 0
            for row_requests in target_category_requests.iterrows():
                # print(row_requests)
                hours_needed = row_requests[1]['Requested Hours']
                # print(hours_needed)
                # print(f"request name: {row_requests[1]['Request Name']}, name: {row_requests[1]['Risk Mitigation Resource: Full Name']}")
                target_cat_abstractor_assignments_real_merged.reset_index(drop=True, inplace=True)
                target_cat_abstractor_assignments_real_merged['Score'] = target_cat_abstractor_assignments_real_merged.apply(
                    lambda row: score_row(row, 'Wrkd_12__c', hours_needed, row_requests[1]['Request Name'], 
                                          row_requests[1]['Risk Mitigation Resource: Full Name'], 
                                          already_shifting=assignment_hours_already_shifted['abstractor_name'].values), axis=1)

                # Sort the dataframe by score in descending order
                target_cat_abstractor_assignments_real_merged_sorted = \
                    target_cat_abstractor_assignments_real_merged.sort_values('Score', ascending=False)
                
                    
                # get the first row of the sorted dataframe and determine if the 'Wrkd_12__c' 
                # is greater than or equal to the hours needed
                
                while (hours_needed > 0) and (iteration < len(target_cat_abstractor_assignments_real_merged_sorted)):
                    # print(f'hours needed: {hours_needed}')
                    first_row = target_cat_abstractor_assignments_real_merged_sorted.iloc[[iteration]].copy()
                    # print(f'first_row: {first_row.columns}')
                    first_row.loc[:, 'Wrkd_12__c'] = np.floor(first_row['Wrkd_12__c'])
                    first_row.loc[:, 'Wrkd_4__c'] = np.floor(first_row['Wrkd_4__c'])
                    hours_available = first_row['Wrkd_12__c'].iloc[0]
                    first_row.loc[:, 'New Project Name'] = row_requests[1]['Team Name']
                    first_row.loc[:, 'Hours Requested'] = row_requests[1]['Requested Hours']
                    first_row = first_row.rename(columns={'Name': 'Old Project Name',
                                                          'Project_Category__c': 'Backfill Project Category'})
                    first_row.loc[:, 'New Assignment Hours'] = min(hours_available, hours_needed)
                    first_row_sel = first_row[['CBIZ_Name', 'Wrkd_4__c', 'Wrkd_12__c', 'Hours Requested', 'New Assignment Hours',
                                                'New Project Name', 'Backfill Project Category', 'Old Project Name']].copy()
                    # print(first_row_sel)
                    try:
                        redundant_capacity_df = pd.concat([redundant_capacity_df, first_row_sel])
                    except:
                        redundant_capacity_df = first_row_sel
                    # add this assignment to the dataframe of assignments that have already been shifted.
                    # get the data in the correct format
                    first_row_to_already_shifted = first_row[['Id', 'CBIZ_Name', 
                                            'Old Project Name', 'New Project Name']]\
                                            .rename(columns={
                                                'Id': 'id',
                                                'CBIZ_Name': 'abstractor_name',
                                                'Old Project Name': 'old_project',
                                                'New Project Name': 'New Project'
                                            }).copy()
                    first_row_to_already_shifted.loc[:, 'id'] = first_row_to_already_shifted['id'].str.strip()
                    # if the abstractor can either cover the assignment or partially cover the assignment, 
                    # then subtract from hours needed and iterate to the next abstractor
                    if hours_available <= hours_needed:
                        # add this assignment to the dataframe of assignments that have already been shifted.
                        assignment_hours_already_shifted = pd.concat([assignment_hours_already_shifted, first_row_to_already_shifted])
                        # reduce the 'Wrkd_12__c' to 0
                        target_cat_abstractor_assignments_real_merged_sorted.iloc[
                            iteration, list(target_cat_abstractor_assignments_real_merged_sorted.columns).index('Wrkd_12__c')] = 0
                        iteration += 1
                    # however, if the abstractor can cover the assignment and then have extra hours left over, 
                    # they can move on to the next request without iterating
                    elif hours_available >= (hours_needed + 2):
                        target_cat_abstractor_assignments_real_merged_sorted.iloc[
                            iteration, 
                            list(target_cat_abstractor_assignments_real_merged_sorted.columns).index('Wrkd_12__c')] = \
                            first_row['Wrkd_12__c'].iloc[0] - hours_needed

                        hours_available -= hours_needed
                    # otherwise just move on to the next abstractor
                    else:
                        # add this assignment to the dataframe of assignments that have already been shifted.
                        assignment_hours_already_shifted = pd.concat([assignment_hours_already_shifted, first_row_to_already_shifted])
                        # reduce the 'Wrkd_12__c' to 0
                        target_cat_abstractor_assignments_real_merged_sorted.iloc[
                            iteration, list(target_cat_abstractor_assignments_real_merged_sorted.columns).index('Wrkd_12__c')] = 0
                        iteration += 1
                    # we always need to decrement the hours needed by the hours available
                    hours_needed -= first_row['Wrkd_12__c'].iloc[0]
                # done with the while loop, so lets get the scores for the abstractors that were used for this request
                redundant_options = target_cat_abstractor_assignments_real_merged_sorted[['Id', 'CBIZ_Name', 'Wrkd_4__c', 
                                                                                            'Wrkd_12__c', 'Full Time Status',
                                                                                            'Project_Category__c', 'Score']].copy()
                redundant_options.loc[:, 'Request Name'] = row_requests[1]['Request Name'][0]
                redundant_options.loc[:, 'New Team Name'] = row_requests[1]['Team Name']
                try:
                    all_redundant_options_df = pd.concat([all_redundant_options_df, redundant_options.head(10)])
                except:
                    all_redundant_options_df = redundant_options.head(10)
            


            # Now that we have the dataframe of the assignments that need to be shifted, 
            # we need to find the abstractors with capacity that can backfill those assignments
            
            if len(redundant_capacity_df) > 0:
                
                for redundant_cap_row in redundant_capacity_df.iterrows():
                    # print(redundant_cap_row[1])
                    replacement_hours_needed = redundant_cap_row[1]['New Assignment Hours']
                    abstractors_with_capacity_fil = pci_sts_acs[
                        (pci_sts_acs['Capticket Hours'] > 0.5) & 
                        (pci_sts_acs['current_skills'].str.contains(redundant_cap_row[1]['Backfill Project Category']))].copy()
                    abstractors_with_capacity_fil = abstractors_with_capacity_fil.sort_values(by = ['Resource_Start_Date', 'Capticket Hours'], ascending=[True, False])
                    abstractors_with_capacity_fil.reset_index(drop=True, inplace=True)
                    abstractors_with_capacity_fil['Score'] = abstractors_with_capacity_fil.apply(
                        lambda redundant_cap_row: score_row(redundant_cap_row, 'Capticket Hours', replacement_hours_needed, row_requests[1]['Request Name'], 
                                          row_requests[1]['Risk Mitigation Resource: Full Name']), axis=1)
                    
                    abstractors_with_capacity_fil = abstractors_with_capacity_fil.sort_values('Score', ascending=False)
                                            
                    redundant_cap_row[1]['Backfill Abstractor'] = ''
                    iteration_capacity = 0
                    while (replacement_hours_needed > 0) & (iteration_capacity < 10):
                        # print(f'replacement hours needed: {replacement_hours_needed}')
                        first_row_capacity = abstractors_with_capacity_fil.iloc[[iteration_capacity]]
                        first_row_capacity.loc[:, 'Capticket Hours'] = np.floor(first_row_capacity['Capticket Hours'])
                        if first_row_capacity['Capticket Hours'].iloc[0] <= replacement_hours_needed:
                            if len(redundant_cap_row[1]['Backfill Abstractor']) == 0:
                                redundant_cap_row[1]['Backfill Abstractor'] = first_row_capacity['CBIZ_Name'].iloc[0] + f"({first_row_capacity['Capticket Hours'].iloc[0]})"
                            else:
                                redundant_cap_row[1]['Backfill Abstractor'] = redundant_cap_row[1]['Backfill Abstractor'] + ', ' +\
                                    first_row_capacity['CBIZ_Name'].iloc[0] + f"({first_row_capacity['Capticket Hours'].iloc[0]})"
                            # abstractors_with_capacity_fil.iloc[
                            #     0, list(abstractors_with_capacity_fil.columns).index('Capticket Hours')] = 0
                            # reduce the hours available in the pci_sts_acs dataframe for this abstractor
                            pci_sts_acs.loc[pci_sts_acs['CBIZ_Name'] == first_row_capacity['CBIZ_Name'].iloc[0], 'Capticket Hours'] = 0
                            iteration_capacity += 1
                        elif first_row_capacity['Capticket Hours'].iloc[0] >= (replacement_hours_needed + 2):
                            if len(redundant_cap_row[1]['Backfill Abstractor']) == 0:
                                redundant_cap_row[1]['Backfill Abstractor'] = first_row_capacity['CBIZ_Name'].iloc[0]
                            else:
                                redundant_cap_row[1]['Backfill Abstractor'] = redundant_cap_row[1]['Backfill Abstractor'] + ', ' +\
                                    first_row_capacity['CBIZ_Name'].iloc[0]
                            abstractors_with_capacity_fil.iloc[
                                0, list(abstractors_with_capacity_fil.columns).index('Capticket Hours')] = \
                                    first_row_capacity['Capticket Hours'].iloc[0] - replacement_hours_needed
                            # reduce the hours available in the pci_sts_acs dataframe for this abstractor
                            pci_sts_acs.loc[pci_sts_acs['CBIZ_Name'] == first_row_capacity['CBIZ_Name'].iloc[0], 'Capticket Hours'] -= replacement_hours_needed
                        else:
                            if len(redundant_cap_row[1]['Backfill Abstractor']) == 0:
                                redundant_cap_row[1]['Backfill Abstractor'] = first_row_capacity['CBIZ_Name'].iloc[0]
                            else:
                                redundant_cap_row[1]['Backfill Abstractor'] = redundant_cap_row[1]['Backfill Abstractor'] + ', ' +\
                                    first_row_capacity['CBIZ_Name'].iloc[0]
                            # reduce the hours available in the pci_sts_acs dataframe for this abstractor
                            # abstractors_with_capacity_fil.iloc[
                            #     0, list(abstractors_with_capacity_fil.columns).index('Capticket Hours')] = 0
                            pci_sts_acs.loc[pci_sts_acs['CBIZ_Name'] == first_row_capacity['CBIZ_Name'].iloc[0], 'Capticket Hours'] = 0
                            iteration_capacity += 1
                        replacement_hours_needed -= first_row_capacity['Capticket Hours'].iloc[0]
                        # print(redundant_cap_row) #[1]['Backfill Abstractor']
                        try:
                            final_redundant_capacity_df = pd.concat([final_redundant_capacity_df, redundant_cap_row[1].to_frame().T])
                        except:
                            final_redundant_capacity_df = redundant_cap_row[1].to_frame().T
                    # done with the while loop, so lets get the scores for the abstractors that were used for this request
                    backfill_options = abstractors_with_capacity_fil[['CBIZ_Name', 'Capticket Hours', 'Resource_Start_Date', 'Full Time Status',
                                                                      'Paid Onboarding Program', 'Score', 'current_skills', 'Capacity_Planned']].copy()
                    backfill_options.loc[:, 'Backfill Project'] = redundant_cap_row[1]['Old Project Name']
                    backfill_options.loc[:, 'Backfill Project Hours'] = redundant_cap_row[1]['New Assignment Hours']
                    try:
                        all_backfill_options_df = pd.concat([all_backfill_options_df, backfill_options.head(10)])
                    except:
                        all_backfill_options_df = backfill_options.head(10)
            else:
                print(f'there were no abstractors available for redundant capacity')
                
        else:
            print(f'After filtering, no assignments in {target_category} met the criteria')

    else:
        print(f'No abstractors in Tr_cats also have the skill in {target_category}')
    if len(final_redundant_capacity_df) > 0:
        final_redundant_capacity_df.to_excel(writer, sheet_name = f'{target_category}_distr', index=False)
        all_redundant_options_df.to_excel(writer, sheet_name = f'{target_category}_dis_op', index=False)
        all_backfill_options_df.to_excel(writer, sheet_name = f'{target_category}_BF_op', index=False)
    # if we are all out of capacity then break the loop
    if len(pci_sts_acs[pci_sts_acs['Capticket Hours'] > 0.5]) == 0:
        print(f'There are no abstractors with capacity left, and we were on {target_category}')
        break
writer.close()

create_capbase.adjust_workbook_column_widths(f"Daily Tools/Redundant_Capacity_Planning_{today}.xlsx")
assignment_hours_already_shifted.to_csv(f'Temp_assignments_already_shifted.csv', index=False)
final_redundant_capacity_df

In [ ]:
abstractors_with_capacity_fil